<font size=4, color=red>**Training_script-v1.0**</font>

- [Vision_encoder](#Vision_model)
- [Text_encoder](#Text_model)
- [Unet_model](#Unet_model)



**TODO Lists**  
- [ ] Running the whole procedure using random data  
- [ ] Baseline  
- [ ] Achieve the dataloader  
- [ ] Try to train and eval the result  

In [1]:
# !pip install -q --upgrade diffusers transformers accelerate
import yaml
import math
import os
import timm
import logging
from collections import OrderedDict
from typing import Tuple, Union

import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from torch import nn

import datasets
import diffusers
import transformers
# from diffusers import UNet2DConditionModel

from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer, CLIPVisionModel

from diffusers.configuration_utils import ConfigMixin
from diffusers.models.cross_attention import AttnProcessor
# from diffusers.models.embeddings import TimestepEmbedding, Timesteps


from model import UNet2DConditionModel

from torchinfo import summary
import open_clip

## Vision_model

In [2]:
down_dir = '/root/.cache/clip/'
open_clip.list_models()



['coca_base',
 'coca_roberta-ViT-B-32',
 'coca_ViT-B-32',
 'coca_ViT-L-14',
 'convnext_base',
 'convnext_base_w',
 'convnext_base_w_320',
 'convnext_large',
 'convnext_large_d',
 'convnext_large_d_320',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge',
 'convnext_xxlarge',
 'convnext_xxlarge_320',
 'mt5-base-ViT-B-32',
 'mt5-xl-ViT-H-14',
 'RN50',
 'RN50-quickgelu',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'RN101',
 'RN101-quickgelu',
 'roberta-ViT-B-32',
 'swin_base_patch4_window7_224',
 'ViT-B-16',
 'ViT-B-16-plus',
 'ViT-B-16-plus-240',
 'ViT-B-32',
 'ViT-B-32-plus-256',
 'ViT-B-32-quickgelu',
 'ViT-bigG-14',
 'ViT-e-14',
 'ViT-g-14',
 'ViT-H-14',
 'ViT-H-16',
 'ViT-L-14',
 'ViT-L-14-280',
 'ViT-L-14-336',
 'ViT-L-16',
 'ViT-L-16-320',
 'ViT-M-16',
 'ViT-M-16-alt',
 'ViT-M-32',
 'ViT-M-32-alt',
 'ViT-S-16',
 'ViT-S-16-alt',
 'ViT-S-32',
 'ViT-S-32-alt',
 'vit_medium_patch16_gap_256',
 'vit_relpos_medium_patch16_cls_224',
 'xlm-roberta-base-ViT-B-32',
 'xlm-roberta-large-ViT-H-14']

In [ ]:
clip_model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32-quickgelu', pretrained='laion400m_e32')

/opt/conda/lib/python3.8/site-packages/open_clip/pretrained.py:301: UserWarning: /root/.cache/clip/vit_b_32-quickgelu-laion400m_e32-46683a32.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(f"{download_target} exists, but the SHA256 checksum does not match; re-downloading the file")
 25%|█████████▍                           | 154M/605M [50:25<4:41:40, 26.7kiB/s]

In [120]:
# from transformers import CLIPVisionModel, CLIPVisionConfig
# from transformers import AutoProcessor, CLIPModel, CLIPVisionModelWithProjection

# from PIL import Image
# import requests
# vision_config = {
#     'hidden_size': 768,
#     'intermediate_size': 3072,
#     'image_size': 224,
#     'patch_size': 32,
# }
# ## create vision model
# model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
# processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

# inputs = processor(images=image, return_tensors="pt")

# outputs = model(**inputs)
# pixel_lhs = outputs.last_hidden_state ## [B,S,D]
# pooled_output = outputs.pooler_output  # pooled CLS states [B, D]

## Text_model

In [132]:
# from transformers import CLIPTextModel, CLIPTextModelWithProjection
# from transformers import AutoProcessor, CLIPModel, AutoTokenizer
# import warnings

# warnings.filterwarnings('ignore')
# text_encoder = CLIPTextModel.from_pretrained('openai/clip-vit-base-patch32')

# tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# inputs = tokenizer(["a photo of a cat", "a photo of a dog"], padding=True, return_tensors="pt")

# outputs = text_encoder(**inputs) 
# text_lhs = outputs.last_hidden_state # [B, S, D]
# pooled_output = outputs.pooler_output


## Unet_model

In [ ]:
# for k in list(unet_ckpt.keys()):
#     if k.split('.')[-2] in ['time_emb_proj', 'time_emb']:
#         unet_ckpt.pop(k)

In [40]:
unet = UNet2DConditionModel(cross_attention_dim=768)
unet_ckpt = torch.load('ckpt/unet_model.bin')   
unet.load_state_dict(unet_ckpt, strict=False)

inp = torch.randn(1,3,64,64)
tx_state = torch.randn(1,77,768)

opt = unet(inp, tx_state)

In [43]:
opt.sample.shape

torch.Size([1, 3, 64, 64])

## Train-Phrase

In [50]:
import argparse
import os
import ruamel_yaml as yaml
import numpy as np
import random
import time
import datetime
import json
from pathlib import Path

import utils
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import torch.distributed as dist

# from models.model_vqa import ALBEF
# from models.vit import interpolate_pos_embed
# from models.tokenization_bert import BertTokenizer


# from dataset.utils import save_result
# from dataset import create_dataset, create_sampler, create_loader, vqa_collate_fn

from scheduler import create_scheduler
from optim import create_optimizer
## tensorboard
from torch.utils.tensorboard import SummaryWriter

In [ ]:
def train(model, data_loader, optimizer, tokenizer, epoch, warmup_steps, device, scheduler, config, Writer):

    # train
    model.train()  
    
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    metric_logger.add_meter('loss', utils.SmoothedValue(window_size=1, fmt='{value:.4f}'))

    header = 'Train Epoch: [{}]'.format(epoch)
    print_freq = 50    
    step_size = 100
    warmup_iterations = warmup_steps*step_size  
    
    for i,(image, question, answer, weights, n) in enumerate(metric_logger.log_every(data_loader, print_freq, header)):
        image, weights = image.to(device,non_blocking=True), weights.to(device,non_blocking=True)      
        question_input = tokenizer(question, padding='longest', truncation=True, max_length=25, return_tensors="pt").to(device) 
        answer_input = tokenizer(answer, padding='longest', return_tensors="pt").to(device) 
        
        if epoch>0 or not config['warm_up']:
            alpha = config['alpha']
        else:
            alpha = config['alpha']*min(1,i/len(data_loader))

        loss = model(image, question_input, answer_input, train=True, alpha=alpha, k=n, weights=weights)        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        
        metric_logger.update(loss=loss.item())
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
     
        if epoch==0 and i%step_size==0 and i<=warmup_iterations: 
            scheduler.step(i//step_size) 
            
    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger.global_avg())   
    ## Extra added
    if dist.get_rank() == 0:
        Writer.add_scalar('LOSS', getattr(metric_logger, 'loss'), i+epoch)
        Writer.add_scalar('lr', optimizer.param_groups[0]['lr'], i+epoch)
        
    return {k: "{:.3f}".format(meter.global_avg) for k, meter in metric_logger.meters.items()} 


@torch.no_grad()
def evaluation(model, data_loader, tokenizer, device, config) :
    # test
    model.eval()
            
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Generate VQA test result:'
    print_freq = 50
    
    result = []
    
    answer_list = [answer+config['eos'] for answer in data_loader.dataset.answer_list]
    answer_input = tokenizer(answer_list, padding='longest', return_tensors='pt').to(device)    
        
    for n, (image, question, question_id) in enumerate(metric_logger.log_every(data_loader, print_freq, header)):        
        image = image.to(device,non_blocking=True)             
        question_input = tokenizer(question, padding='longest', return_tensors="pt").to(device)        

        topk_ids, topk_probs = model(image, question_input, answer_input, train=False, k=config['k_test'])      
        
        for ques_id, topk_id, topk_prob in zip(question_id, topk_ids, topk_probs):
            ques_id = int(ques_id.item())          
            _, pred = topk_prob.max(dim=0)
            result.append({"question_id":ques_id, "answer":data_loader.dataset.answer_list[topk_id[pred]]})   

    return result


def main(args, config):
    ## distribued_init
    utils.init_distributed_mode(args)    
    
    device = torch.device(args.device)
    ## Writer
    Writer = None
    if dist.get_rank ==0 :
        Writer = SummaryWriter(config['tensorboard_dir'])
        
    # fix the seed for reproducibility
    seed = args.seed + utils.get_rank()
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    cudnn.benchmark = True
    
    start_epoch = 0
    max_epoch = config['schedular']['epochs']
    warmup_steps = config['schedular']['warmup_epochs']
    
    
    #### Dataset #### 
    print("Creating vqa datasets")
    datasets = create_dataset('vqa', config)   
    
    if args.distributed:
        num_tasks = utils.get_world_size()
        global_rank = utils.get_rank()            
        samplers = create_sampler(datasets, [True, False], num_tasks, global_rank)         
    else:
        samplers = [None, None]
    
    train_loader, test_loader = create_loader(datasets,samplers,
                                              batch_size=[config['batch_size_train'],config['batch_size_test']],
                                              num_workers=[4,4],is_trains=[True, False], 
                                              collate_fns=[vqa_collate_fn,None]) 

    tokenizer = BertTokenizer.from_pretrained(args.text_encoder)

    #### Model #### 
    print("Creating model")
    model = ALBEF(config=config, text_encoder=args.text_encoder, text_decoder=args.text_decoder, tokenizer=tokenizer)
    model = model.to(device)   
    
    arg_opt = utils.AttrDict(config['optimizer'])
    optimizer = create_optimizer(arg_opt, model)
    arg_sche = utils.AttrDict(config['schedular'])
    lr_scheduler, _ = create_scheduler(arg_sche, optimizer)          
        
    if args.checkpoint:    
        checkpoint = torch.load(args.checkpoint, map_location='cpu') 
        state_dict = checkpoint['model']
        
        # reshape positional embedding to accomodate for image resolution change
        pos_embed_reshaped = interpolate_pos_embed(state_dict['visual_encoder.pos_embed'],model.visual_encoder)         
        state_dict['visual_encoder.pos_embed'] = pos_embed_reshaped   
        
        if not args.evaluate:
            if config['distill']:
                m_pos_embed_reshaped = interpolate_pos_embed(state_dict['visual_encoder_m.pos_embed'],model.visual_encoder_m)   
                state_dict['visual_encoder_m.pos_embed'] = m_pos_embed_reshaped 
                
            for key in list(state_dict.keys()):
                if 'bert' in key:
                    encoder_key = key.replace('bert.','')         
                    state_dict[encoder_key] = state_dict[key] 
                # intialize text decoder as multimodal encoder (last 6 layers of model.text_encoder)    
                if 'text_encoder' in key:                
                    if 'layer' in key:
                        encoder_keys = key.split('.')
                        layer_num = int(encoder_keys[4])
                        if layer_num<6:
                            del state_dict[key]  
                            continue
                        else:
                            decoder_layer_num = (layer_num-6)
                            encoder_keys[4] = str(decoder_layer_num)
                            encoder_key = '.'.join(encoder_keys)     
                    else:
                        encoder_key = key
                    decoder_key = encoder_key.replace('text_encoder','text_decoder')  
                    state_dict[decoder_key] = state_dict[key]     

                    del state_dict[key]                
                
        msg = model.load_state_dict(state_dict,strict=False)  
        print('load checkpoint from %s'%args.checkpoint)
        print(msg)  

        
    model_without_ddp = model
    if args.distributed:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
        model_without_ddp = model.module    
    
    
    print("Start training")
    start_time = time.time()

    for epoch in range(start_epoch, max_epoch):
        if epoch>0:
            lr_scheduler.step(epoch+warmup_steps)  
        
        if not args.evaluate:
            if args.distributed:
                train_loader.sampler.set_epoch(epoch)

            train_stats = train(model, train_loader, optimizer, tokenizer, epoch, warmup_steps, device, lr_scheduler, config, Writer)  ## Add Writer

        if args.evaluate:
            break
            
        if utils.is_main_process():               
            log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                         'epoch': epoch,
                        }                
            with open(os.path.join(args.output_dir, "log.txt"),"a") as f:
                f.write(json.dumps(log_stats) + "\n")                        
                         
            save_obj = {
                'model': model_without_ddp.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'config': config,
                'epoch': epoch,
            }
            torch.save(save_obj, os.path.join(args.output_dir, 'checkpoint_%02d.pth'%epoch))  

        dist.barrier()   
  
    vqa_result = evaluation(model, test_loader, tokenizer, device, config)        
    result_file = save_result(vqa_result, args.result_dir, 'vqa_result_epoch%d'%epoch)
                     
    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str)) 
    
            

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--config', default='./configs/VQA.yaml') 
    parser.add_argument('--checkpoint', default='') 
    parser.add_argument('--output_dir', default='output/vqa')
    parser.add_argument('--evaluate', action='store_true')    
    parser.add_argument('--text_encoder', default='bert-base-uncased')
    parser.add_argument('--text_decoder', default='bert-base-uncased')
    parser.add_argument('--device', default='cuda')
    parser.add_argument('--seed', default=42, type=int)
    parser.add_argument('--world_size', default=1, type=int, help='number of distributed processes')    
    parser.add_argument('--dist_url', default='env://', help='url used to set up distributed training')
    parser.add_argument('--distributed', default=True, type=bool)
    ## ADD
#     parser.add_argument('--tensorboard_dir', type=str, default='./tensenboard/')
    args = parser.parse_args()

    config = yaml.load(open(args.config, 'r'), Loader=yaml.Loader)

    args.result_dir = os.path.join(args.output_dir, 'result')

    Path(args.output_dir).mkdir(parents=True, exist_ok=True)
    Path(args.result_dir).mkdir(parents=True, exist_ok=True)
        
    yaml.dump(config, open(os.path.join(args.output_dir, 'config.yaml'), 'w'))
    
    main(args, config)